
# Build a real-world agent
Next, build a practical weather forecasting agent that demonstrates key production concepts:
Detailed system prompts for better agent behavior
Create tools that integrate with external data
Model configuration for consistent responses
Structured output for predictable results
Conversational memory for chat-like interactions
Create and run the agent create a fully functional agent
Let’s walk through each step:

1
Define the system prompt

The system prompt defines your agent’s role and behavior. Keep it specific and actionable:

In [9]:
from langchain.agents import create_agent
import os
from langchain.chat_models import init_chat_model
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
import getpass

In [10]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

2
Create tools

Tools let a model interact with external systems by calling functions you define. Tools can depend on runtime context and also interact with agent memory.
Notice below how the get_user_location tool uses runtime context:

In [11]:
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

3
Configure your model

Set up your language model with the right parameters for your use case:

In [12]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

model = init_chat_model("gpt-4.1", temperature=0.5, max_retries=3, timeout=60, max_tokens=1000, streaming=True)

4
Define response format

Optionally, define a structured response format if you need the agent responses to match a specific schema.

In [13]:
# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

5
Add memory

Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.

In [14]:
checkpointer = InMemorySaver()

6
Create and run the agent

Now assemble your agent with all the components and run it!

In [15]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like Florida is serving up its signature dish—sunshine with a side of vitamin sea! It's always sunny in Florida, so sunglasses are practically a dress code. Stay cool, sun-chaser!", weather_conditions="It's always sunny in Florida!")
ResponseFormat(punny_response="You're welcome! If you ever need another weather update, just give me a shout—I'm always ready to sprinkle a little sunshine on your day. Have a weather-ful time!", weather_conditions=None)
